In [2]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [25]:
# read data from file:
merge_train0_no0 = pd.read_csv('C:/data/Kaggle_ASHRAE_energy/merge_train0_no0.csv')
merge_train0_no0.head()

,site_id,building_id,0,1,2,3,4,5,6,7,...,airtemp,sealev,dewtemp,windsp,North,West,South,nowind,dewair,meter_reading
0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.6980,0.6070,0.8640,0.4255,0.0,0.0,0.0,0.0,0.9463,68.220
1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.7060,0.5970,0.8735,0.2356,0.0,0.0,0.0,0.0,0.9470,81.424
2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.7407,0.5957,0.8820,0.1488,0.0,0.0,0.0,0.0,0.9307,70.020
3,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.7630,0.5900,0.8735,0.2107,1.0,0.0,0.0,0.0,0.9146,70.621
4,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.7344,0.5850,0.8916,0.2356,1.0,0.0,0.0,0.0,0.9390,74.022


In [26]:
merge_train0_no0.shape

(11486383, 29)

In [45]:
merge_train0_no0.drop(['site_id','building_id','timestamp','meter_reading'], axis=1).shape

(11486383, 25)

In [39]:
X = [merge_train0_no0.building_id.values, merge_train0_no0.drop(['site_id','building_id','timestamp','meter_reading'], axis=1).values]
y = merge_train0_no0.meter_reading.values

In [63]:
def build_model():
    inputs = []
    embeddings = []
    
    input_building_id = tf.keras.layers.Input(shape=(1,), name='input_building_id')
    embedding = tf.keras.layers.Embedding(1449, 40, input_length=1, name = 'building_id_emb')(input_building_id)
    embedding = tf.keras.layers.Flatten()(embedding)
    embedding = tf.keras.layers.Dropout(0.1)(embedding)
    embedding = tf.keras.layers.Dense(20, activation='relu')(embedding)
    embedding = tf.keras.layers.Dropout(0.05)(embedding)
    embedding = tf.keras.layers.BatchNormalization()(embedding)
    inputs.append(input_building_id)
    embeddings.append(embedding)
    
    input_numeric = tf.keras.layers.Input(shape=(25,), name = 'input_numeric')
#    embedding_num = tf.keras.layers.Dense(35)(input_numeric)
    inputs.append(input_numeric)
    embeddings.append(input_numeric)
    
    main_Layers = tf.keras.layers.Concatenate()(embeddings)
    
    main_Layers = tf.keras.layers.Dense(50, activation='relu')(main_Layers)
    main_Layers = tf.keras.layers.BatchNormalization()(main_Layers)
    main_Layers = tf.keras.layers.Dropout(0.05)(main_Layers)
    main_Layers = tf.keras.layers.Dense(20, activation='relu')(main_Layers)
    main_Layers = tf.keras.layers.BatchNormalization()(main_Layers)
    main_Layers = tf.keras.layers.Dropout(0.05)(main_Layers)
    main_Layers = tf.keras.layers.Dense(10, activation='relu')(main_Layers)
    main_Layers = tf.keras.layers.BatchNormalization()(main_Layers)
#    main_Layer = tf.keras.layers.Dropout(0.05)(main_Layer)
    output = tf.keras.layers.Dense(1, activation='relu')(main_Layers)

    model = tf.keras.Model(inputs, output)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001,beta_1=0.9,beta_2=0.999, epsilon=1e-08), 
                  loss=tf.keras.losses.mean_squared_error
    #              , metrics=['mae']
                 )
    return model

In [64]:
model1 = build_model()

In [65]:
model1.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_building_id (InputLayer)  [(None, 1)]          0                                            
__________________________________________________________________________________________________
building_id_emb (Embedding)     (None, 1, 40)        57960       input_building_id[0][0]          
__________________________________________________________________________________________________
flatten_4 (Flatten)             (None, 40)           0           building_id_emb[0][0]            
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 40)           0           flatten_4[0][0]                  
____________________________________________________________________________________________

In [ ]:
history = model1.fit(X, y, epochs=1
                     , batch_size=64
#                     , verbose=0
                    )

Train on 11486383 samples
 1551936/11486383 [===>..........................] - ETA: 14:38 - loss: nan

In [51]:
model_check = model1

In [52]:
model_check.get_config()

{'name': 'model_2',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 1),
    'dtype': 'float32',
    'sparse': False,
    'name': 'input_5'},
   'name': 'input_5',
   'inbound_nodes': []},
  {'class_name': 'Embedding',
   'config': {'name': 'embedding_3',
    'trainable': True,
    'batch_input_shape': (None, None),
    'dtype': 'float32',
    'input_dim': 1449,
    'output_dim': 20,
    'embeddings_initializer': {'class_name': 'RandomUniform',
     'config': {'minval': -0.05, 'maxval': 0.05, 'seed': None}},
    'embeddings_regularizer': None,
    'activity_regularizer': None,
    'embeddings_constraint': None,
    'mask_zero': False,
    'input_length': None},
   'name': 'embedding_3',
   'inbound_nodes': [[['input_5', 0, 0, {}]]]},
  {'class_name': 'Flatten',
   'config': {'name': 'flatten_2',
    'trainable': True,
    'dtype': 'float32',
    'data_format': 'channels_last'},
   'name': 'flatten_2',
   'inbound_nodes': [[['embedding_3', 0, 0, {}]]]},